<a href="https://colab.research.google.com/github/UBGidado/My_Research/blob/Notebooks/DialogRE_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets spacy
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 4.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import json
import os
import torch
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from typing import List, Dict, Tuple, Any
import re
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import pickle
import logging

### Load Dataset

In [3]:
import json
import logging
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Configure logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# File paths
data_path = "/content/drive/MyDrive/My Research/dialogre/data_v2/en/data"

train_file = f"{data_path}/train.json"
dev_file = f"{data_path}/dev.json"
test_file = f"{data_path}/test.json"

# Load from JSON data
def load_dialogre_data(file_path):
    """
    Load DialogRE data from the official JSON format and convert it
    into a standard dictionary structure for processing.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    processed_conversations = []

    for conv_idx, conversation in enumerate(raw_data):
        # Each conversation is a list: [dialogue_turns, relation_instances]
        if len(conversation) != 2:
            logger.warning(
                f"Conversation {conv_idx} has unexpected structure: {len(conversation)} elements"
            )
            continue

        dialogue_turns = conversation[0]
        relation_instances = conversation[1]

        # Convert dialogue turns to standard format
        formatted_dialogue = []
        for turn_idx, turn in enumerate(dialogue_turns):
            if isinstance(turn, dict):
                formatted_dialogue.append(turn)
            else:
                # Handle "Speaker: Text" format
                if isinstance(turn, str) and ':' in turn:
                    speaker, text = turn.split(':', 1)
                    formatted_dialogue.append({
                        'speaker': speaker.strip(),
                        'text': text.strip(),
                        'turn': turn_idx
                    })
                else:
                    formatted_dialogue.append({
                        'speaker': f"Speaker_{turn_idx}",
                        'text': str(turn),
                        'turn': turn_idx
                    })

        # Create conversation dictionary
        conv_dict = {
            'dialogue': formatted_dialogue,
            'relations': relation_instances,
            'id': f"conv_{conv_idx}",
            'x': [],  # Placeholder: can be populated later
            'y': []   # Placeholder: can be populated later
        }

        processed_conversations.append(conv_dict)

    logger.info(f"Loaded {len(processed_conversations)} conversations from {file_path}")
    return processed_conversations

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_data = load_dialogre_data(train_file)
dev_data = load_dialogre_data(dev_file)
test_data = load_dialogre_data(test_file)

### Model Setup

In [5]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define special tokens for entity marking
SPECIAL_TOKENS = {
    'E1_START': '[E1]',
    'E1_END': '[/E1]',
    'E2_START': '[E2]',
    'E2_END': '[/E2]',
    'SPEAKER_SEP': '[SPEAKER]'
}

# Add special tokens to tokenizer
tokenizer.add_special_tokens({'additional_special_tokens': list(SPECIAL_TOKENS.values())})

print("✓ Libraries imported and tokenizer initialized")
print(f"Tokenizer vocabulary size: {len(tokenizer)}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


✓ Libraries imported and tokenizer initialized
Tokenizer vocabulary size: 30527


### Entity and Speaker Processing

In [6]:
ner_model_name = "dslim/bert-base-NER"  # High-accuracy model for Person extraction
ner_tokenizer = AutoTokenizer.from_pretrained(ner_model_name)
ner_model = AutoModelForTokenClassification.from_pretrained(ner_model_name)
ner_pipeline = pipeline("ner", model=ner_model, tokenizer=ner_tokenizer, aggregation_strategy="simple")

# Load spaCy model for coreference resolution
nlp = spacy.load("en_core_web_lg")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


#### Entity Processing

In [7]:
def extract_entities_with_advanced_ner(conversation: Dict) -> List[Dict]:
    """
    Extract PERSON entities with their positions using transformer NER model.
    Returns list of entities with text, positions, speaker, and confidence.
    """
    all_entities = []

    for turn_idx, utterance in enumerate(conversation['dialogue']):
        text = utterance['text']
        speaker = utterance['speaker']

        # Use NER pipeline to process utterance text
        try:
            ner_results = ner_pipeline(text)
            # Filter for only PERSON entities with confidence > 0.8
            person_entities = [entity for entity in ner_results
                             if entity['entity_group'] == 'PER' and entity['score'] > 0.8]

            for entity in person_entities:
                all_entities.append({
                    'text': entity['word'].strip(),
                    'start': entity['start'],
                    'end': entity['end'],
                    'speaker': speaker,
                    'utterance_idx': turn_idx,
                    'confidence': round(entity['score'], 4),
                    'source_utterance': text
                })
        except Exception as e:
            print(f"Error processing utterance: {text}\nError: {e}")
            continue

    return all_entities

def extract_entities_with_spacy(conversation: Dict) -> List[Dict]:
    """
    Extract entities using spaCy's built-in NER and rule-based matching.
    This helps capture additional entity mentions and provides linguistic features.
    """
    spacy_entities = []

    for turn_idx, utterance in enumerate(conversation['dialogue']):
        text = utterance['text']
        speaker = utterance['speaker']

        # Process with spaCy
        doc = nlp(text)

        for ent in doc.ents:
            if ent.label_ in ['PERSON', 'ORG']:  # Include organizations as they can be social entities
                spacy_entities.append({
                    'text': ent.text.strip(),
                    'start': ent.start_char,
                    'end': ent.end_char,
                    'speaker': speaker,
                    'utterance_idx': turn_idx,
                    'confidence': 0.9,  # spaCy doesn't provide confidence, use default high value
                    'source_utterance': text,
                    'label': ent.label_
                })

    return spacy_entities

def resolve_entities_spacy(conversation: Dict) -> List[Dict]:
    """
    Use spaCy's processing to get additional linguistic information and
    basic coreference through noun chunks and dependency parsing.
    """
    entity_info = []

    # Process each utterance with spaCy
    for turn_idx, utterance in enumerate(conversation['dialogue']):
        text = utterance['text']
        speaker = utterance['speaker']
        doc = nlp(text)

        # Extract noun chunks that might refer to entities
        for chunk in doc.nouns:
            if any(token.ent_type_ in ['PERSON', 'ORG'] for token in chunk):
                entity_info.append({
                    'text': chunk.text,
                    'speaker': speaker,
                    'utterance_idx': turn_idx,
                    'type': 'noun_chunk',
                    'root_text': chunk.root.text
                })

    return entity_info

def canonicalize_entities(raw_entities: List[Dict], spacy_entities: List[Dict]) -> Dict[str, Any]:
    """
    Create canonical entity mapping using combined NER results from both models.
    """
    canonical_entities = {}
    mention_to_canonical = {}

    # Combine entities from both sources
    all_entities = raw_entities + spacy_entities

    # First pass: use exact string matching for canonicalization
    for entity in all_entities:
        entity_text = entity['text']

        # Check if this is a new entity or a mention of an existing one
        found_canonical = None
        for canonical_name in canonical_entities:
            # Simple matching: exact match or contained within
            if (entity_text.lower() == canonical_name.lower() or
                entity_text.lower() in canonical_name.lower() or
                canonical_name.lower() in entity_text.lower()):
                found_canonical = canonical_name
                break

        if found_canonical:
            canonical_name = found_canonical
        else:
            canonical_name = entity_text
            canonical_entities[canonical_name] = {
                'mentions': [],
                'source_utterances': [],
                'speakers': set(),
                'confidence_scores': [],
                'types': set()
            }

        # Add this mention to the canonical entity
        canonical_entities[canonical_name]['mentions'].append({
            'text': entity_text,
            'utterance_idx': entity['utterance_idx'],
            'speaker': entity['speaker'],
            'confidence': entity.get('confidence', 0.8)
        })
        canonical_entities[canonical_name]['source_utterances'].append(entity['source_utterance'])
        canonical_entities[canonical_name]['speakers'].add(entity['speaker'])
        canonical_entities[canonical_name]['confidence_scores'].append(entity.get('confidence', 0.8))
        if 'label' in entity:
            canonical_entities[canonical_name]['types'].add(entity['label'])

        mention_to_canonical[entity_text] = canonical_name

    return {
        'canonical_entities': canonical_entities,
        'mention_to_canonical_map': mention_to_canonical
    }

#### Speaker Processing and Analysis

In [8]:
def process_speaker_information(conversation: Dict, canonical_entities: Dict) -> Dict[str, Any]:
    """
    Process speaker information and analyze speaker-entity relationships.
    """
    speakers = set()
    speaker_utterances = defaultdict(list)
    speaker_entity_mentions = defaultdict(lambda: defaultdict(int))

    for turn_idx, utterance in enumerate(conversation['dialogue']):
        speaker = utterance['speaker']
        text = utterance['text']
        speakers.add(speaker)

        # Record utterance
        speaker_utterances[speaker].append({
            'turn': turn_idx,
            'text': text,
            'length': len(text.split()),
            'has_question': '?' in text
        })

        # Count entity mentions per speaker using our canonical entities
        for canonical_name in canonical_entities:
            # Simple string matching for mention detection
            if canonical_name.lower() in text.lower():
                speaker_entity_mentions[speaker][canonical_name] += 1

    return {
        'speakers': list(speakers),
        'speaker_utterances': dict(speaker_utterances),
        'speaker_entity_mentions': dict(speaker_entity_mentions),
        'num_speakers': len(speakers),
        'conversation_length': len(conversation['dialogue'])
    }

# Cell 5: Master Processing Function
def process_conversation_entities(conversation: Dict) -> Dict[str, Any]:
    """
    Master function to process all entities and speakers in a conversation.
    """
    # Extract entities using both transformer NER and spaCy
    raw_entities = extract_entities_with_advanced_ner(conversation)
    spacy_entities = extract_entities_with_spacy(conversation)
    spacy_linguistic = resolve_entities_spacy(conversation)

    # Canonicalize entities
    entity_data = canonicalize_entities(raw_entities, spacy_entities)

    # Process speaker information
    speaker_data = process_speaker_information(conversation, entity_data['canonical_entities'])

    # Compile comprehensive results
    return {
        'conversation_id': conversation.get('id', 'unknown'),
        'raw_ner_entities': raw_entities,
        'spacy_entities': spacy_entities,
        'spacy_linguistic_info': spacy_linguistic,
        **entity_data,
        **speaker_data,
        'processing_stats': {
            'bert_entities_count': len(raw_entities),
            'spacy_entities_count': len(spacy_entities),
            'canonical_entities_count': len(entity_data['canonical_entities']),
            'speakers_count': speaker_data['num_speakers']
        }
    }

# Cell 6: Example Usage and Validation
def validate_processing(dialogre_data: List[Dict], sample_index: int = 0):
    """
    Validate the processing pipeline on a sample conversation.
    """
    sample_conversation = dialogre_data[sample_index]
    print("=== Processing Sample Conversation ===")
    print(f"Conversation ID: {sample_conversation.get('id', 'unknown')}")
    print(f"Number of utterances: {len(sample_conversation['dialogue'])}")

    # Process the conversation
    results = process_conversation_entities(sample_conversation)

    print(f"\n=== Results ===")
    print(f"BERT entities: {len(results['raw_ner_entities'])}")
    print(f"spaCy entities: {len(results['spacy_entities'])}")
    print(f"Canonical Entities: {len(results['canonical_entities'])}")

    print(f"\nTop entities:")
    for i, (entity, info) in enumerate(list(results['canonical_entities'].items())[:5]):
        print(f"  {i+1}. {entity}: {len(info['mentions'])} mentions")

    print(f"\nSpeakers: {results['speakers']}")

    return results

##  Relation Extraction and Entity Pair Creation

In [9]:
def extract_relation_data(conversation: Dict) -> List[Dict]:
    """Extract relation information from conversation"""
    relations = []

    for rel_instance in conversation.get('relations', []):
        x_entity = rel_instance.get('x', '')
        y_entity = rel_instance.get('y', '')
        relation_types = rel_instance.get('r', [])

        for relation_type in relation_types:
            relations.append({
                'subject': x_entity,
                'object': y_entity,
                'relation': relation_type,
                'conversation_id': conversation.get('id', 'unknown')
            })

    return relations

def create_entity_pairs(conversation: Dict) -> List[Dict]:
    """Create all possible entity pairs for relation classification"""
    entity_pairs = []

    # Extract all unique entities from relation instances
    all_entities = set()
    entity_idx_map = {}  # Map entity name to index

    for rel_instance in conversation.get('relations', []):
        x_entity = rel_instance.get('x', '')
        y_entity = rel_instance.get('y', '')

        if x_entity and x_entity not in entity_idx_map:
            entity_idx_map[x_entity] = len(entity_idx_map)
        if y_entity and y_entity not in entity_idx_map:
            entity_idx_map[y_entity] = len(entity_idx_map)

    # Create conversation entities lists
    conversation['x'] = list(entity_idx_map.keys())
    conversation['y'] = list(entity_idx_map.keys())

    # Get actual relations for positive examples
    actual_relations = {}
    for rel_instance in conversation.get('relations', []):
        x_entity = rel_instance.get('x', '')
        y_entity = rel_instance.get('y', '')
        relation_types = rel_instance.get('r', [])

        if x_entity in entity_idx_map and y_entity in entity_idx_map:
            x_idx = entity_idx_map[x_entity]
            y_idx = entity_idx_map[y_entity]

            # For simplicity, take the first relation type if multiple exist
            if relation_types:
                actual_relations[(x_idx, y_idx)] = relation_types[0]

    # Create pairs between all entities
    entity_names = list(entity_idx_map.keys())
    for i, entity_x in enumerate(entity_names):
        for j, entity_y in enumerate(entity_names):
            if i == j:
                continue  # Skip self-relations if desired

            pair_key = (i, j)
            relation_type = actual_relations.get(pair_key, 'no_relation')

            entity_pairs.append({
                'subject': entity_x,
                'object': entity_y,
                'relation': relation_type,
                'subject_idx': i,
                'object_idx': j,
                'is_positive': relation_type != 'no_relation',
                'conversation_id': conversation.get('id', 'unknown')
            })

    return entity_pairs

### Text Processing and Entity Marking

In [10]:
def mark_entities_in_text(text: str, subject: str, object: str,
                         subject_pos: Tuple[int, int] = None,
                         object_pos: Tuple[int, int] = None) -> str:
    """Mark entities in text with special tokens"""

    if subject_pos and object_pos:
        # Use provided positions
        s_start, s_end = subject_pos
        o_start, o_end = object_pos

        # Sort positions to handle overlapping cases
        positions = [(s_start, s_end, 'subject'), (o_start, o_end, 'object')]
        positions.sort(key=lambda x: x[0])

        # Apply marking from right to left to preserve positions
        marked_text = text
        for start, end, entity_type in reversed(positions):
            if entity_type == 'subject':
                marked_text = (marked_text[:start] +
                             SPECIAL_TOKENS['E1_START'] + marked_text[start:end] +
                             SPECIAL_TOKENS['E1_END'] + marked_text[end:])
            else:
                marked_text = (marked_text[:start] +
                             SPECIAL_TOKENS['E2_START'] + marked_text[start:end] +
                             SPECIAL_TOKENS['E2_END'] + marked_text[end:])
    else:
        # Simple string replacement (fallback)
        marked_text = text.replace(subject, f"{SPECIAL_TOKENS['E1_START']}{subject}{SPECIAL_TOKENS['E1_END']}")
        marked_text = marked_text.replace(object, f"{SPECIAL_TOKENS['E2_START']}{object}{SPECIAL_TOKENS['E2_END']}")

    return marked_text

def process_conversation_text(conversation: Dict, entity_pair: Dict) -> str:
    """Process conversation text and mark entities"""
    dialogue_texts = []
    subject = entity_pair['subject']
    object = entity_pair['object']

    # Handle both string and dictionary dialogue turns
    for i, turn in enumerate(conversation['dialogue']):
        if isinstance(turn, dict):
            speaker = turn.get('speaker', 'Unknown')
            text = turn.get('text', '')
        else:
            # Assume it's a string or needs parsing
            speaker = f"Speaker_{i}"
            text = str(turn)

        # Mark entities in this utterance
        marked_text = mark_entities_in_text(text, subject, object)

        # Add speaker information
        processed_text = f"{SPECIAL_TOKENS['SPEAKER_SEP']} {speaker}: {marked_text}"
        dialogue_texts.append(processed_text)

    # Join all utterances
    full_text = " ".join(dialogue_texts)

    return full_text

def prepare_bert_input(text: str, max_length: int = 512) -> Dict:
    """Prepare BERT input from marked text"""
    # Tokenize the text with special tokens
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )

    # Find positions of entity markers
    input_ids = encoding['input_ids'][0].tolist()
    e1_start = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS['E1_START'])
    e1_end = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS['E1_END'])
    e2_start = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS['E2_START'])
    e2_end = tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS['E2_END'])

    # Find entity positions
    entity_positions = {
        'e1_start': input_ids.index(e1_start) if e1_start in input_ids else -1,
        'e1_end': input_ids.index(e1_end) if e1_end in input_ids else -1,
        'e2_start': input_ids.index(e2_start) if e2_start in input_ids else -1,
        'e2_end': input_ids.index(e2_end) if e2_end in input_ids else -1,
    }

    return {
        'input_ids': encoding['input_ids'][0],
        'attention_mask': encoding['attention_mask'][0],
        'token_type_ids': encoding['token_type_ids'][0],
        'entity_positions': entity_positions
    }

###  Main Preprocessing Pipeline

In [11]:
def preprocess_dialogre_dataset_simple(train_data: List[Dict], dev_data: List[Dict], test_data: List[Dict], output_dir: str) -> Dict[str, Any]:
    """Simplified preprocessing pipeline focusing on BERT baseline without augmentation."""

    os.makedirs(output_dir, exist_ok=True)
    logger.info("Starting simplified preprocessing (no augmentation)...")

    # Process each dataset separately
    datasets = {
        'train': train_data,
        'dev': dev_data,
        'test': test_data
    }

    all_processed_samples = []
    relation_labels = set()

    for dataset_name, data in datasets.items():
        logger.info(f"Processing {dataset_name} set with {len(data)} conversations...")

        dataset_samples = []
        for conv_idx, conversation in enumerate(data):
            try:
                # Extract entity pairs for this conversation
                entity_pairs = create_entity_pairs(conversation)

                for pair in entity_pairs:
                    # Process conversation text
                    processed_text = process_conversation_text(conversation, pair)

                    # Prepare BERT input
                    bert_input = prepare_bert_input(processed_text)

                    # Create processed sample
                    sample = {
                        'conversation_id': conversation.get('id', f'unknown_{conv_idx}'),
                        'dataset': dataset_name,
                        'subject': pair['subject'],
                        'object': pair['object'],
                        'relation': pair['relation'],
                        'is_positive': pair['is_positive'],
                        'processed_text': processed_text,
                        'input_ids': bert_input['input_ids'],
                        'attention_mask': bert_input['attention_mask'],
                        'token_type_ids': bert_input['token_type_ids'],
                        'entity_positions': bert_input['entity_positions']
                    }

                    dataset_samples.append(sample)
                    relation_labels.add(pair['relation'])

            except Exception as e:
                logger.error(f"Error processing conversation {conv_idx} in {dataset_name}: {e}")
                continue

        # Save this dataset split
        dataset_file = os.path.join(output_dir, f'{dataset_name}_samples.pkl')
        with open(dataset_file, 'wb') as f:
            pickle.dump(dataset_samples, f)

        logger.info(f"Saved {len(dataset_samples)} samples for {dataset_name}")
        all_processed_samples.extend(dataset_samples)

    # Create label mapping
    relation_to_id = {rel: idx for idx, rel in enumerate(sorted(relation_labels))}
    id_to_relation = {idx: rel for rel, idx in relation_to_id.items()}

    # Add label IDs to all samples
    for sample in all_processed_samples:
        sample['label_id'] = relation_to_id[sample['relation']]

    # Save combined data (without augmentation)
    output_data = {
        'samples': all_processed_samples,
        'relation_to_id': relation_to_id,
        'id_to_relation': id_to_relation,
        'num_relations': len(relation_labels),
        'num_samples': len(all_processed_samples)
    }

    output_file = os.path.join(output_dir, 'bert_baseline_data.pkl')
    with open(output_file, 'wb') as f:
        pickle.dump(output_data, f)

    # Save metadata
    metadata = {
        'num_relations': len(relation_labels),
        'num_samples': len(all_processed_samples),
        'relation_labels': list(relation_labels),
        'relation_to_id': relation_to_id,
        'augmentation_applied': False
    }

    metadata_file = os.path.join(output_dir, 'metadata.json')
    with open(metadata_file, 'w') as f:
        json.dump(metadata, f, indent=2)

    logger.info(f"BERT baseline preprocessing completed! Total samples: {len(all_processed_samples)}")
    return output_data

### Train/Validation/Test Split

In [12]:
def create_train_val_test_split_simple(processed_data: Dict[str, Any],
                                      output_dir: str,
                                      train_ratio: float = 0.7,
                                      val_ratio: float = 0.15,
                                      test_ratio: float = 0.15) -> None:
    """Create simple train/validation/test splits without stratification"""

    samples = processed_data['samples']

    # Simple random split without stratification
    train_samples, temp_samples = train_test_split(
        samples,
        test_size=(val_ratio + test_ratio),
        random_state=42
    )

    val_samples, test_samples = train_test_split(
        temp_samples,
        test_size=test_ratio/(val_ratio + test_ratio),
        random_state=42
    )

    # Save splits (same as above)
    splits = {
        'train': train_samples,
        'validation': val_samples,
        'test': test_samples
    }

    for split_name, split_data in splits.items():
        split_file = os.path.join(output_dir, f'{split_name}_split.pkl')
        with open(split_file, 'wb') as f:
            pickle.dump(split_data, f)

        print(f"{split_name} split: {len(split_data)} samples")

    print("Data splits created successfully (non-stratified)!")

In [13]:
# DATA_PATH = train_data  # Update this - this variable is no longer needed as we pass the data directly
OUTPUT_DIR = "processed_data"

# Process the data
processed_data = preprocess_dialogre_dataset_simple(
    train_data=train_data,
    dev_data=dev_data,
    test_data=test_data,
    output_dir=OUTPUT_DIR
)

# Create splits - USE THE FIXED VERSION
create_train_val_test_split_simple(processed_data, OUTPUT_DIR)

# Print summary
print(f"\nPreprocessing Summary:")
print(f"- Total samples: {processed_data['num_samples']}")
print(f"- Number of relation types: {processed_data['num_relations']}")
print(f"- Output directory: {OUTPUT_DIR}")

# Check the rare classes file to see what was removed
rare_classes_file = os.path.join(OUTPUT_DIR, 'rare_classes_info.json')
if os.path.exists(rare_classes_file):
    with open(rare_classes_file, 'r') as f:
        rare_info = json.load(f)
    print(f"- Removed {len(rare_info['removed_relations'])} rare relation types")

train split: 34294 samples
validation split: 7349 samples
test split: 7349 samples
Data splits created successfully (non-stratified)!

Preprocessing Summary:
- Total samples: 48992
- Number of relation types: 38
- Output directory: processed_data


### Model Setup

In [14]:
import torch
import torch.nn as nn
from transformers import BertModel, BertPreTrainedModel

class BertForRelationExtraction(BertPreTrainedModel):
    """BERT model for relation extraction with entity attention - Optimized version"""

    def __init__(self, config, num_labels):
        super().__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config)

        # Classification layer
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size * 2, num_labels)

        # Initialize weights
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                e1_positions=None, e2_positions=None, labels=None):

        # Get BERT outputs
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )

        sequence_output = outputs.last_hidden_state  # [batch_size, seq_len, hidden_size]

        # Get representations for the two entities (vectorized approach)
        e1_reps = self._get_entity_representation_vectorized(sequence_output, e1_positions)
        e2_reps = self._get_entity_representation_vectorized(sequence_output, e2_positions)

        # Concatenate entity representations
        combined_reps = torch.cat([e1_reps, e2_reps], dim=-1)
        combined_reps = self.dropout(combined_reps)

        # Classification
        logits = self.classifier(combined_reps)

        # Calculate loss if labels are provided
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

        return {'loss': loss, 'logits': logits}

    def _get_entity_representation_vectorized(self, sequence_output, entity_positions):
        """Vectorized version of entity representation extraction"""
        batch_size, seq_len, hidden_size = sequence_output.shape
        device = sequence_output.device

        # Extract start positions for all samples at once
        start_positions = entity_positions[:, 0]  # [batch_size]

        # Use advanced indexing to get start token representations
        batch_indices = torch.arange(batch_size, device=device)

        # Clamp positions to valid range
        start_positions = torch.clamp(start_positions, 0, seq_len - 1)

        # Get representations at start positions
        entity_reps = sequence_output[batch_indices, start_positions]  # [batch_size, hidden_size]

        return entity_reps

    def _get_entity_representation(self, sequence_output, entity_positions):
        """Fallback method using loops (same as above but kept for reference)"""
        batch_size, seq_len, hidden_size = sequence_output.shape
        device = sequence_output.device

        entity_reps = torch.zeros(batch_size, hidden_size, device=device, dtype=sequence_output.dtype)

        for i in range(batch_size):
            start_pos = entity_positions[i, 0].item()
            end_pos = entity_positions[i, 1].item()

            start_pos = max(0, min(start_pos, seq_len - 1))
            end_pos = max(start_pos + 1, min(end_pos, seq_len))

            entity_span = sequence_output[i, start_pos:end_pos]
            entity_rep = entity_span.mean(dim=0)
            entity_reps[i] = entity_rep

        return entity_reps


### Create DataLoaders for Training

In [15]:
from torch.utils.data import Dataset, DataLoader
import torch

class RelationDataset(Dataset):
    """PyTorch Dataset for relation extraction"""

    def __init__(self, samples, tokenizer, max_length=128):
        self.samples = samples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]

        # Convert to tensors
        return {
            'input_ids': torch.tensor(sample['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(sample['attention_mask'], dtype=torch.long),
            'token_type_ids': torch.tensor(sample['token_type_ids'], dtype=torch.long),
            'e1_positions': torch.tensor([sample['entity_positions']['e1_start'],
                                        sample['entity_positions']['e1_end']], dtype=torch.long),
            'e2_positions': torch.tensor([sample['entity_positions']['e2_start'],
                                        sample['entity_positions']['e2_end']], dtype=torch.long),
            'labels': torch.tensor(sample['label_id'], dtype=torch.long),
            'relation': sample['relation'],
            'is_positive': sample['is_positive']
        }

def create_data_loaders(train_samples, val_samples, test_samples, batch_size=32):
    """Create DataLoader objects for each split"""

    train_dataset = RelationDataset(train_samples, tokenizer)
    val_dataset = RelationDataset(val_samples, tokenizer)
    test_dataset = RelationDataset(test_samples, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader

## Training Loop

In [21]:
from tqdm import tqdm
from transformers import BertConfig
from sklearn.metrics import f1_score


def initialize_bert_model(num_labels, tokenizer):
    """Properly initialize BERT model for relation extraction"""

    # Load configuration
    config = BertConfig.from_pretrained('bert-base-uncased')
    config.num_labels = num_labels

    # Initialize model
    model = BertForRelationExtraction.from_pretrained(
        'bert-base-uncased',
        config=config,
        num_labels=num_labels,
        ignore_mismatched_sizes=True
    )

    # Resize token embeddings AFTER model initialization
    model.bert.resize_token_embeddings(len(tokenizer))

    # Move to device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Verify model is properly initialized
    for name, param in model.named_parameters():
        if param.device.type == 'meta':
            raise RuntimeError(f"Parameter {name} is still on meta device after initialization!")

    print(f"Model initialized successfully on {device}")
    print(f"Model has {sum(p.numel() for p in model.parameters())} parameters")

    return model, device


# Updated training function
def train_bert_model_fixed(train_loader, val_loader, num_labels, tokenizer, learning_rate=2e-5, num_epochs=5):
    """Fixed training function that properly handles model initialization"""

    # Initialize model properly
    model, device = initialize_bert_model(num_labels, tokenizer)

    # Create optimizer after model is on correct device
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    # Training loop
    best_f1 = 0
    train_losses = []
    val_f1_scores = []

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch + 1}/{num_epochs}")

        # Training phase
        model.train()
        total_train_loss = 0
        num_batches = 0

        from tqdm import tqdm
        train_pbar = tqdm(train_loader, desc="Training")

        for batch in train_pbar:
            # Move batch to device (only tensor items)
            batch_tensors = {}
            for k, v in batch.items():
                if isinstance(v, torch.Tensor):
                    batch_tensors[k] = v.to(device)
                else:
                    batch_tensors[k] = v  # Keep non-tensor items as is

            # Forward pass
            outputs = model(
                input_ids=batch_tensors['input_ids'],
                attention_mask=batch_tensors['attention_mask'],
                token_type_ids=batch_tensors['token_type_ids'],
                e1_positions=batch_tensors['e1_positions'],
                e2_positions=batch_tensors['e2_positions'],
                labels=batch_tensors['labels']
            )

            loss = outputs['loss']

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Track loss
            loss_value = loss.item()
            total_train_loss += loss_value
            num_batches += 1
            train_pbar.set_postfix({'loss': loss_value})

        avg_train_loss = total_train_loss / num_batches
        train_losses.append(avg_train_loss)

        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                batch_tensors = {}
                for k, v in batch.items():
                    if isinstance(v, torch.Tensor):
                        batch_tensors[k] = v.to(device)

                outputs = model(
                    input_ids=batch_tensors['input_ids'],
                    attention_mask=batch_tensors['attention_mask'],
                    token_type_ids=batch_tensors['token_type_ids'],
                    e1_positions=batch_tensors['e1_positions'],
                    e2_positions=batch_tensors['e2_positions']
                )

                logits = outputs['logits']
                preds = torch.argmax(logits, dim=-1)

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(batch_tensors['labels'].cpu().numpy())

        # Calculate F1 score
        from sklearn.metrics import f1_score
        val_f1 = f1_score(all_labels, all_preds, average='macro')
        val_f1_scores.append(val_f1)

        print(f"Epoch {epoch + 1}: Train Loss = {avg_train_loss:.4f}, Val F1 = {val_f1:.4f}")

        # Save best model
        if val_f1 > best_f1:
            best_f1 = val_f1
            torch.save({
                'model_state_dict': model.state_dict(),
                'epoch': epoch,
                'best_val_f1': best_f1
            }, 'best_bert_model.pth')
            print(f"New best model saved with F1: {best_f1:.4f}")

    return model, train_losses, val_f1_scores

## Evaluation and Inference

In [22]:
         def evaluate_model(model, test_loader, id_to_relation):
    """Evaluate the trained model on test set"""

    model.eval()
    all_preds = []
    all_labels = []
    all_relations = []
    all_texts = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            # Store text and relations for analysis
            all_relations.extend(batch['relation'])
            # Note: You might want to store the actual text too

            # Move tensors to device
            batch_tensors = {k: v.to(model.device) for k, v in batch.items()
                           if k not in ['relation', 'is_positive']}

            outputs = model(**batch_tensors)
            logits = outputs['logits']
            preds = torch.argmax(logits, dim=-1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(batch_tensors['labels'].cpu().numpy())

    # Convert back to relation labels
    pred_relations = [id_to_relation[pred] for pred in all_preds]
    true_relations = [id_to_relation[label] for label in all_labels]

    # Generate classification report
    print("\n" + "="*50)
    print("CLASSIFICATION REPORT")
    print("="*50)
    print(classification_report(true_relations, pred_relations, zero_division=0))

    # Calculate macro F1 score
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Macro F1 Score: {macro_f1:.4f}")

    return pred_relations, true_relations, macro_f1

## Execute the Full Pipeline

In [ ]:
# Load your processed data
processed_file = os.path.join(OUTPUT_DIR, 'bert_baseline_data.pkl')
with open(processed_file, 'rb') as f:
    processed_data = pickle.load(f)

# Load the splits
train_file = os.path.join(OUTPUT_DIR, 'train_split.pkl')
val_file = os.path.join(OUTPUT_DIR, 'validation_split.pkl')
test_file = os.path.join(OUTPUT_DIR, 'test_split.pkl')

with open(train_file, 'rb') as f:
    train_samples = pickle.load(f)
with open(val_file, 'rb') as f:
    val_samples = pickle.load(f)
with open(test_file, 'rb') as f:
    test_samples = pickle.load(f)

print(f"Train samples: {len(train_samples)}")
print(f"Validation samples: {len(val_samples)}")
print(f"Test samples: {len(test_samples)}")

# Set your desired batch size here
BATCH_SIZE = 16  # ← ADJUST THIS NUMBER (8, 16, 32, etc.)

# Create DataLoaders with your chosen batch size
train_loader, val_loader, test_loader = create_data_loaders(
    train_samples,
    val_samples,
    test_samples,
    batch_size=32
)

print(f"Using batch size: {BATCH_SIZE}")
print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")


# Train the model
print("Starting training...")
model, train_losses, val_f1_scores = train_bert_model_fixed(
    train_loader,
    val_loader,
    num_labels=processed_data['num_relations'],
    tokenizer=tokenizer,
    learning_rate=2e-5,
    num_epochs=3
)

# Evaluate on test set
print("\nEvaluating on test set...")
pred_relations, true_relations, test_f1 = evaluate_model(
    model, test_loader, processed_data['id_to_relation']
)

print(f"\n=== BERT BASELINE RESULTS ===")
print(f"Test Macro F1: {test_f1:.4f}")
print("Training completed!")

Train samples: 34294
Validation samples: 7349
Test samples: 7349
Using batch size: 16
Train batches: 1072
Val batches: 230
Starting training...


Some weights of BertForRelationExtraction were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model initialized successfully on cuda
Model has 109544486 parameters

Epoch 1/3


Training:   0%|          | 0/1072 [00:00<?, ?it/s]/tmp/ipython-input-3431103970.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(sample['input_ids'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(sample['attention_mask'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(sample['token_type_ids'], dtype=torch.long),
Validation: 100%|██████████

Epoch 1: Train Loss = 0.8422, Val F1 = 0.0414
New best model saved with F1: 0.0414

Epoch 2/3


Training:   0%|          | 0/1072 [00:00<?, ?it/s]/tmp/ipython-input-3431103970.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(sample['input_ids'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(sample['attention_mask'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(sample['token_type_ids'], dtype=torch.long),
Validation: 100%|██████████

Epoch 2: Train Loss = 0.6251, Val F1 = 0.1319
New best model saved with F1: 0.1319

Epoch 3/3


Training:   0%|          | 0/1072 [00:00<?, ?it/s]/tmp/ipython-input-3431103970.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'input_ids': torch.tensor(sample['input_ids'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'attention_mask': torch.tensor(sample['attention_mask'], dtype=torch.long),
/tmp/ipython-input-3431103970.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  'token_type_ids': torch.tensor(sample['token_type_ids'], dtype=torch.long),
Training:  33%|███▎      | 